Soundcloud comments, especially on the most popular tracks, are pretty obnoxious. There's a toxic mix of self-promotion and links to external phishing sites.

Can we use similar techniques from email spam filtering techniques to classify what kind of comments are spam?

First, let's grab some comments from a popular track. I'm going to attempt to manually label them using the soundcloud community guidelines: https://soundcloud.com/community-guidelines I'll be interpreting them pretty liberally to include any blatant self-promotion or external phishing/marketing sites.

In [77]:
import time
import soundcloud

# create a client object with your app credentials
client = soundcloud.Client(client_id='85d3857863f87e52fc6f3a48e11dead7')

slippery = client.get('/resolve', url="https://soundcloud.com/migosatl/slippery-feat-gucci-mane")

304747566

In [78]:
next_comments = client.get('/tracks/' + str(slippery.id) + '/comments', linked_partitioning = 1)
final_comment_list = next_comments.obj['collection']
final_page = False
while not final_page:
    next_comments = client.get(next_comments.obj['next_href'][26:])
    final_comment_list = final_comment_list + next_comments.obj['collection']
    if not 'next_href' in next_comments.keys():
        final_page = True

Well, I would call the majority of these comments pretty low-quality. However, I'm not into broad censorship, so I'm only interested in removing the that are blatant spam.

But first let's put these comments into a pandas dataframe. I'm also going to import the timestamp, which I think might be useful to the model, and we can examine everything later.

In [79]:
import pandas as pd
train_comments = pd.DataFrame([[comment_obj['id'],comment_obj['body'],
                                comment_obj['timestamp'],slippery_track.obj['duration']]
                               for comment_obj in final_comment_list],
                               columns=['id','body','timestamp','track_length'])

In [80]:
train_comments['spam'] = 0
train_comments.to_csv("unlabeled_comments.csv")

There's a _lot_ of self promotion in these comments sections. However, I think that people casually posting a link or two to their music isn't a clear violation of the community guidelines and is actually part of what makes soundcloud a unique community. Thus, here are the guidelines I used when marking these comments as spam:

1. The user is promoting their music and posted 3 or more links in one comment or 3 comments with any number of links.
2. The user is posting chain mail messages (you must post or you will have bad luck, etc.).
3. The user is posting obvious links to pornography (check out this girl bro, etc.).
4. The user is posting links to referral scams (I bought a red ferrarri and made 70 dollars per hour...,etc.).

Here is the dataframe of labeled comments.

In [84]:
train_comments = pd.read_excel("labeled_comments.xlsx")
train_comments[train_comments['spam']==1]

,id,body,timestamp,track_length,spam
5,335100258,Don't read this. You will be kissed on the nea...,205366,304145,1
7,335088864,MASTAPIECE Are The Next Aritst Out of Houston!...,184776,304145,1
9,335067382,Don't read this. You will be kissed on the nea...,69779,304145,1
32,334896296,MASTAPIECE Are The Next Aritst Out of Houston!...,181146,304145,1
45,334719340,my mom recently got a very cool Lincoln MKZ Hy...,152321,304145,1
53,334643893,MASTAPIECE Are The Next Aritst Out of Houston!...,126605,304145,1
59,334571651,just as Bryan said I'm shocked that a student ...,61087,304145,1
60,334571592,just as Bryan said I'm shocked that a student ...,69974,304145,1
61,334571489,just as Bryan said I'm shocked that a student ...,302353,304145,1
62,334571232,just as Bryan said I'm shocked that a student ...,87471,304145,1
